In [4]:
import yfinance as yf 
from yahoo_fin.stock_info import *
## Standard Python Data Science stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import math
import statsmodels.api as sm
import datetime as dt
import math
from scipy.optimize import minimize
from scipy.stats import (norm as norm, linregress as linregress)
plt.rcParams['figure.figsize'] = [20, 10]
#from SHARPR_backend import *
from fredapi import Fred
fred = Fred(api_key='3cc2743ce40daec36ca56954fefedca7')
from FedTools import MonetaryPolicyCommittee
from FedTools import BeigeBooks
from FedTools import FederalReserveMins
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from math import ceil

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table = pd.read_html(url, header=0)
sp500_df = table[0]  
sp500_tickers = sp500_df['Symbol'].tolist()

url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
table = pd.read_html(url, header=0)
dow_df = table[2]  
dow_tickers = dow_df['Symbol'].tolist()

url = "https://en.wikipedia.org/wiki/Nasdaq-100"
table = pd.read_html(url,header=0)
nasdaq_df = table[4] 
nasdaq_tickers = nasdaq_df.Symbol.to_list()

tickers = set(sp500_tickers) | set(dow_tickers) | set(nasdaq_tickers)

In [8]:
prices = [] 
symbols = []
start_date = "2010-01-01"
for symbol in tickers:
    df = yf.download(symbol, start=start_date)['Adj Close']
    if not df.empty:
        prices.append(df)
        symbols.append(symbol)
all_prices = pd.concat(prices, axis=1)
all_prices.columns = symbols

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
benchmark = yf.download('SPY', start = '2010-01-01')['Adj Close']
benchmark.columns = ['Benchmark']

risk_free = yf.download('BIL', start = '2010-01-01')['Adj Close']
risk_free.columns = ['Risk Free Return']

all_m_ret = all_prices.pct_change(fill_method=None).resample('ME').agg(lambda x: (x + 1).prod() - 1).dropna()
all_w_ret = all_prices.pct_change(fill_method=None).resample('W').agg(lambda x: (x + 1).prod() - 1).dropna()

benchmark_m = benchmark.pct_change(fill_method=None).resample('ME').agg(lambda x: (x + 1).prod() - 1).dropna()
benchmark_w = benchmark.pct_change(fill_method=None).resample('W').agg(lambda x: (x + 1).prod() - 1).dropna()

risk_free_m = risk_free.pct_change(fill_method=None).resample('ME').agg(lambda x: (x + 1).prod() - 1).dropna()
risk_free_w = risk_free.pct_change(fill_method=None).resample('W').agg(lambda x: (x + 1).prod() - 1).dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
all_m_ret.to_csv("all_monthly_returns.csv")
all_w_ret.to_csv("all_weekly_returns.csv")

In [9]:
all_m_ret = pd.read_csv("all_monthly_returns.csv", index_col=0, parse_dates=True)
all_m_ret.index = pd.to_datetime(all_m_ret.index)
all_w_ret = pd.read_csv("all_weekly_returns.csv", index_col=0, parse_dates=True)
all_w_ret.index = pd.to_datetime(all_w_ret.index)

In [21]:
def rolling_cumulative_returns(returns, window):
    return (returns.add(1).rolling(window=window).apply(np.prod, raw=True) - 1)

def top_n_columns_by_last_row(cum_rets, n):
    last_row = cum_rets.iloc[-1]
    top_columns = last_row.nlargest(n).index.tolist()
    return top_columns

def top_n_columns_by_row(cum_rets, n, row_num):
    last_row = cum_rets.iloc[row_num]
    top_columns = last_row.nlargest(n).index.tolist()
    return top_columns

def top_n_columns_by_all_rows(cum_rets, n):
    top_columns_df = pd.DataFrame(
        index=cum_rets.index,
        columns=[f"Top_{i+1}" for i in range(n)])
    for idx, row in cum_rets.iterrows():
        top_columns_df.loc[idx] = row.nlargest(n).index.tolist()
    return top_columns_df


def triple_momentum_filter(returns,lookback_periods = (12,8,4),top_counts = (50,25,5),holding_length=1):

    roll_let_long = rolling_cumulative_returns(returns,lookback_periods[0]).iloc[(lookback_periods[0]-1):-holding_length]
    roll_let_medium = rolling_cumulative_returns(returns,lookback_periods[1]).iloc[(lookback_periods[0]-1):-holding_length]
    roll_let_short = rolling_cumulative_returns(returns,lookback_periods[2]).iloc[(lookback_periods[0]-1):-holding_length]

    holding_cum_ret = (returns.iloc[(lookback_periods[0]):-holding_length].add(1).rolling(window=1).apply(np.prod, raw=True) - 1)

    portfolio_formation_dates = roll_let_long.index
    stock_picks = []
    for i in range(len(portfolio_formation_dates)):
        top_long = top_n_columns_by_row(roll_let_long,top_counts[0],row_num=i)
        top_medium = top_n_columns_by_row(roll_let_medium[top_long],top_counts[1],row_num=i)
        top_short = top_n_columns_by_row(roll_let_short[top_medium],top_counts[2],row_num=i)
        stock_picks.append(top_short)
    stock_picks = pd.DataFrame(stock_picks)
    stock_picks.index = portfolio_formation_dates
    stock_picks.columns = [f"Top_{i+1}" for i in range(len(stock_picks.columns))]

    earn_dates = holding_cum_ret.index
    raw_returns = []
    for i in range(len(earn_dates)):
        picks = stock_picks.iloc[i].values
        raw_rets = holding_cum_ret[picks].iloc[i].values
        raw_returns.append(raw_rets)
    raw_returns = pd.DataFrame(raw_returns)
    raw_returns.index = earn_dates
    raw_returns.columns = [f"Top_{i+1}" for i in range(len(raw_returns.columns))]

    strategy_return = raw_returns.mean(axis=1)

    return {
        "Strategy Return": strategy_return,
        "Raw Returns": raw_returns,
        "Stock Picks": stock_picks}

def plot_cumulative_returns(strategy_return, benchmark):
    strat = (strategy_return + 1).cumprod()
    bench = (benchmark + 1).cumprod()

    aligned_strat, aligned_bench = strat.align(bench, join='inner')

    aligned_strat = aligned_strat.squeeze()
    aligned_bench = aligned_bench.squeeze()

    combined_df = pd.DataFrame({
        'Strategy Return': aligned_strat,
        'Benchmark Return': aligned_bench})

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=combined_df.index, 
        y=combined_df['Strategy Return'],
        mode='lines',
        name='Strategy Return',
        line=dict(width=2, color='blue')))

    fig.add_trace(go.Scatter(
        x=combined_df.index, 
        y=combined_df['Benchmark Return'],
        mode='lines',
        name='Benchmark Return',
        line=dict(width=2, color='orange')))

    fig.update_layout(
        title='Cumulative Returns: Strategy vs Benchmark',
        xaxis_title='Date',
        yaxis_title='Cumulative Return',
        legend=dict(font=dict(size=12)),
        template='plotly_white',
        width=800,
        height=600)
    return fig

def plot_rolling_cumulative_return(rolling_cumulative_return):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=rolling_cumulative_return.index,
        y=rolling_cumulative_return.values,
        mode='lines',
        name='Rolling Cumulative Return',
        line=dict(width=2, color='blue')))

    fig.add_trace(go.Scatter(
        x=rolling_cumulative_return.index,
        y=[0] * len(rolling_cumulative_return),
        mode='lines',
        name='Baseline',
        line=dict(width=1, dash='dash', color='black')))

    fig.update_layout(
        title="Rolling 1-year Cumulative Return",
        xaxis_title="Date",
        yaxis_title="Cumulative Return",
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    return fig

def plot_top_stocks(returns, lookbacks, tickers_to_plot):
    tickers_to_plot = [ticker for ticker in tickers_to_plot if ticker in returns.columns]
    data_to_plot = (returns.loc["2024", tickers_to_plot] + 1).cumprod()
    n_cols = 4
    n_rows = ceil(len(data_to_plot.columns) / n_cols)
    fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=data_to_plot.columns)
    lookback_indices = [data_to_plot.index[-lb] for lb in lookbacks if lb <= len(data_to_plot.index)]
    for i, col in enumerate(data_to_plot.columns):
        row = i // n_cols + 1
        col_position = i % n_cols + 1
        series = data_to_plot[col]
        fig.add_trace(go.Scatter(
            x=series.index,
            y=series.values,
            mode='lines',
            name=col,
            legendgroup=col,
            line=dict(width=2)
        ), row=row, col=col_position)
        for lb_index in lookback_indices:
            fig.add_shape(
                type="line",
                x0=lb_index, x1=lb_index,
                y0=series.min(), y1=series.max(),
                line=dict(color="black", dash="dash"),
                row=row, col=col_position)
    fig.update_layout(
        title="Top Stocks Cumulative Returns",
        height=300 * n_rows,
        width=1200,
        template="plotly_white")
    return fig

def dual_momentum_filter(returns, lookback_periods=(12, 6), top_counts=(50, 10), holding_length=1):
    roll_let_long = rolling_cumulative_returns(returns, lookback_periods[0]).iloc[(lookback_periods[0] - 1):-holding_length]
    roll_let_short = rolling_cumulative_returns(returns, lookback_periods[1]).iloc[(lookback_periods[0] - 1):-holding_length]


    holding_cum_ret = (returns.iloc[(lookback_periods[0]):-holding_length].add(1).rolling(window=1).apply(np.prod, raw=True) - 1)
    portfolio_formation_dates = roll_let_long.index
    stock_picks = []

    for i in range(len(portfolio_formation_dates)):
        top_long = top_n_columns_by_row(roll_let_long, top_counts[0], row_num=i)
        top_short = top_n_columns_by_row(roll_let_short[top_long], top_counts[1], row_num=i)
        stock_picks.append(top_short)

    stock_picks = pd.DataFrame(stock_picks)
    stock_picks.index = portfolio_formation_dates
    stock_picks.columns = [f"Top_{i+1}" for i in range(len(stock_picks.columns))]

    earn_dates = holding_cum_ret.index
    raw_returns = []

    for i in range(len(earn_dates)):
        picks = stock_picks.iloc[i].values
        raw_rets = holding_cum_ret[picks].iloc[i].values
        raw_returns.append(raw_rets)

    raw_returns = pd.DataFrame(raw_returns)
    raw_returns.index = earn_dates
    raw_returns.columns = [f"Top_{i+1}" for i in range(len(raw_returns.columns))]

    strategy_return = raw_returns.mean(axis=1)
    return {
        "Strategy Return": strategy_return,
        "Raw Returns": raw_returns,
        "Stock Picks": stock_picks}

def simple_momentum_filter(returns, lookback_period=12, top_count=10, holding_length=1):
    roll_let = rolling_cumulative_returns(returns, lookback_period).iloc[(lookback_period - 1):-holding_length]

    holding_cum_ret = (returns.iloc[(lookback_period):-holding_length]
                       .add(1).rolling(window=1).apply(np.prod, raw=True) - 1)

    portfolio_formation_dates = roll_let.index
    stock_picks = []

    for i in range(len(portfolio_formation_dates)):
        top_stocks = top_n_columns_by_row(roll_let, top_count, row_num=i)
        stock_picks.append(top_stocks)

    stock_picks = pd.DataFrame(stock_picks)
    stock_picks.index = portfolio_formation_dates
    stock_picks.columns = [f"Top_{i+1}" for i in range(len(stock_picks.columns))]

    earn_dates = holding_cum_ret.index
    raw_returns = []

    for i in range(len(earn_dates)):
        picks = stock_picks.iloc[i].values
        raw_rets = holding_cum_ret[picks].iloc[i].values
        raw_returns.append(raw_rets)
    raw_returns = pd.DataFrame(raw_returns)
    raw_returns.index = earn_dates
    raw_returns.columns = [f"Top_{i+1}" for i in range(len(raw_returns.columns))]

    strategy_return = raw_returns.mean(axis=1)

    return {
        "Strategy Return": strategy_return,
        "Raw Returns": raw_returns,
        "Stock Picks": stock_picks}

def get_latest_picks_triple(returns,lookback_periods = (12,8,4),top_counts = (50,25,5)):
    returns = returns.iloc[-(lookback_periods[0]+1):-1]
    roll_let_long = rolling_cumulative_returns(returns,lookback_periods[0])
    roll_let_medium = rolling_cumulative_returns(returns,lookback_periods[1])
    roll_let_short = rolling_cumulative_returns(returns,lookback_periods[2])
    top_long = top_n_columns_by_last_row(roll_let_long,top_counts[0])
    top_medium = top_n_columns_by_last_row(roll_let_medium[top_long],top_counts[1])
    top_short = top_n_columns_by_last_row(roll_let_short[top_medium],top_counts[2])
    return top_short

def get_latest_picks_dual(returns,lookback_periods = (12,4),top_counts = (50,5)):
    returns = returns.iloc[-(lookback_periods[0]+1):-1]
    roll_let_long = rolling_cumulative_returns(returns,lookback_periods[0])
    roll_let_short = rolling_cumulative_returns(returns,lookback_periods[1])
    top_long = top_n_columns_by_last_row(roll_let_long,top_counts[0])
    top_short = top_n_columns_by_last_row(roll_let_short[top_long],top_counts[1])
    return top_short

def get_latest_picks_simple(returns,lookback_period = 12,top_count = 5):
    returns = returns.iloc[-(lookback_period+1):-1]
    roll_let = rolling_cumulative_returns(returns,lookback_period)
    top = top_n_columns_by_last_row(roll_let,top_count)
    return top

def calc_rolling_cum_ret(return_series,freq=52):
    rolling_cumulative_return = ((return_series.add(1).rolling(window=freq).apply(lambda x: x.prod(), raw=True))- 1).dropna()
    return rolling_cumulative_return

In [10]:
backtest = triple_momentum_filter(returns=all_w_ret, lookback_periods=(12,8,4), top_counts=(50,30,10), holding_length=1)

In [11]:
backtest["Stock Picks"]

,Top_1,Top_2,Top_3,Top_4,Top_5,Top_6,Top_7,Top_8,Top_9,Top_10
Date,,,,,,,,,,
2010-03-28,LULU,INCY,LVS,C,GRMN,HST,PCAR,WYNN,AKAM,F
2010-04-04,LULU,INCY,C,LVS,HST,CBRE,DAL,RF,RCL,TSN
2010-04-11,MGM,LULU,LVS,WYNN,CBRE,NFLX,RF,HST,C,TSCO
2010-04-18,URI,LULU,UAL,NFLX,LYV,LVS,C,WYNN,FAST,AOS
2010-04-25,URI,NFLX,HBAN,AIG,MGM,CMG,MAS,MAR,WY,TSCO
...,...,...,...,...,...,...,...,...,...,...
2024-11-24,APP,MSTR,TPL,PLTR,AXON,PAYC,TEAM,TSLA,VST,UAL
2024-12-01,APP,MSTR,PLTR,AXON,TSLA,TPL,TPR,WBD,APO,DDOG
2024-12-08,MSTR,APP,DDOG,PLTR,LULU,TSLA,MRVL,TPR,DIS,NFLX


In [12]:
plot_cumulative_returns(backtest["Strategy Return"], benchmark_w)

In [22]:
rolling_cumulative_return = calc_rolling_cum_ret(backtest["Strategy Return"],freq=52)
plot_rolling_cumulative_return(rolling_cumulative_return)

In [14]:
tickers_to_plot = get_latest_picks_triple(all_w_ret,lookback_periods = (12,8,4),top_counts = (50,30,10))
plot_top_stocks(returns = all_w_ret,lookbacks=(12,8,4),tickers_to_plot = tickers_to_plot)

In [48]:
import streamlit as st

In [55]:
def update_data():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url, header=0)
    sp500_df = table[0]  
    sp500_tickers = sp500_df['Symbol'].tolist()

    url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
    table = pd.read_html(url, header=0)
    dow_df = table[2]  
    dow_tickers = dow_df['Symbol'].tolist()

    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    table = pd.read_html(url,header=0)
    nasdaq_df = table[4] 
    nasdaq_tickers = nasdaq_df.Symbol.to_list()

    tickers = set(sp500_tickers) | set(dow_tickers) | set(nasdaq_tickers)

    prices = [] 
    symbols = []
    start_date = "2010-01-01"

    progress_bar = st.progress(0)
    counter_text = st.empty()
    total_tickers = len(tickers)

    for i, symbol in enumerate(tickers, start=1):
        df = yf.download(symbol, start=start_date)['Adj Close']
        if not df.empty:
            prices.append(df)
            symbols.append(symbol)
        progress_bar.progress(i / total_tickers)
        counter_text.text(f"Processed {i}/{total_tickers} tickers")

    all_prices = pd.concat(prices, axis=1)
    all_prices.columns = symbols

    all_m_ret = all_prices.pct_change(fill_method=None).resample('ME').agg(lambda x: (x + 1).prod() - 1).dropna().iloc[:-1]
    all_w_ret = all_prices.pct_change(fill_method=None).resample('W').agg(lambda x: (x + 1).prod() - 1).dropna().iloc[:-1]

    progress_bar.empty()
    counter_text.empty()

    return all_m_ret, all_w_ret

In [56]:
a, b = update_data()

2024-12-29 23:09:22.286 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-29 23:09:22.287 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-29 23:09:22.288 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-29 23:09:22.289 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
[*********************100%***********************]  1 of 1 completed
2024-12-29 23:09:22.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-29 23:09:22.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-29 23:09:22.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-29 23:09:22.684 Thread 'MainThread': miss

In [58]:
b

,CSX,MSTR,GILD,ROP,MSFT,CSCO,TTD,HD,INTC,ADSK,...,VRTX,GS,CSGP,MCD,PG,ARM,WBD,INTU,WDAY,V
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-10,0.072042,-0.012063,0.028637,-0.000071,-0.009370,-0.001215,0.000000,0.010813,-0.002395,0.022984,...,-0.080696,0.007106,0.002130,-0.014973,-0.011126,0.000000,0.057419,0.008763,0.000000,-0.012934
2010-01-17,-0.044674,0.090421,0.022003,-0.013921,0.006523,-0.010543,0.000000,-0.014148,-0.001441,-0.006855,...,-0.011311,-0.052205,-0.021729,0.007115,0.006287,0.000000,-0.010677,0.012227,0.000000,-0.009885
2010-01-24,-0.118305,-0.063520,0.012302,-0.011603,-0.061569,-0.058606,0.000000,-0.029751,-0.042788,-0.037577,...,-0.001741,-0.067127,-0.019556,0.017823,-0.001265,0.000000,-0.047487,-0.043230,0.000000,-0.039587
2010-01-31,-0.028559,-0.033914,0.047526,-0.020153,-0.026934,-0.021767,0.000000,0.010462,-0.025615,-0.052191,...,-0.043348,-0.035037,-0.005664,-0.015144,0.020561,0.000000,-0.039819,-0.016279,0.000000,-0.008461
2010-02-07,0.001400,-0.155676,-0.039155,0.048722,-0.005678,0.054739,0.000000,-0.001071,0.011636,-0.030265,...,-0.020573,0.036579,-0.046558,0.015057,-0.004062,0.000000,-0.040459,-0.005741,0.000000,0.006461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-24,0.018503,0.238456,0.020249,0.021725,0.004819,0.018970,0.097757,0.028958,0.006160,0.073943,...,-0.031630,0.015568,0.120140,-0.008031,0.039755,0.056397,0.096529,-0.069417,0.032150,0.000904
2024-12-01,0.024882,-0.081563,0.026500,0.011247,0.015492,0.011272,-0.008867,0.021738,-0.018367,-0.091418,...,0.038051,0.009605,0.019171,0.019740,0.016905,-0.012501,0.036597,0.002515,-0.066331,0.016649
2024-12-08,-0.057455,0.019460,-0.004861,-0.026905,0.047490,0.011485,0.082147,0.005220,-0.130145,0.054162,...,-0.012881,-0.009726,-0.049422,0.016109,-0.030347,0.049147,0.017176,0.007386,0.065723,-0.012917
